In [1]:
import datetime
import time
import tweepy
import pandas as pd
import numpy as np 
# import pymongo
import json

from decouple import config


# get_details function

In [2]:

# import pymongo
import json
consumer_key = config('consumer_key') 
consumer_secret = config('consumer_secret') 
access_key = config('access_key') 
access_secret = config('access_secret') 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
def convert_twitter_datetime_to_string(date):
	return time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(date,'%a %b %d %H:%M:%S +0000 %Y'))

def convert_string_to_datetime(date):
	return datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")

def get_details(username):
	
	try:
		userdata=api.get_user(screen_name=username)
		
	except Exception as e:
		return
		# 200 is the max 
	
	end_date = datetime.datetime.now()
	start_date = datetime.datetime.now() - datetime.timedelta(days=7)
	user = userdata._json
	num_of_retweets_by_user = 0
	num_of_tweets_this_week = 0
	monday_to_sunday = [0] * 7
	twelve_am_to_eleven_pm = [0] * 24
	retweeted=0
	most_recent_post=0
	tweet_language=''
	try:
		timeline = api.user_timeline(screen_name=username, count=200,include_rts=True, tweet_mode="extended")
		if len(timeline) >= 1:
			most_recent_post=convert_twitter_datetime_to_string(timeline[0]._json["created_at"])
			tweet_language=timeline[0]._json["lang"]
			for tweet in timeline:
				tweet = tweet._json
				# Number of times this Tweet has been retweeted.
				retweeted+=tweet["retweet_count"]
				# Retweets can be distinguished from typical Tweets by the existence of a retweeted_status
				if ("retweeted_status" in tweet):
					num_of_retweets_by_user += 1
				tweet_time = convert_string_to_datetime(convert_twitter_datetime_to_string(tweet["created_at"]))
				if (tweet_time < end_date and tweet_time > start_date):
					num_of_tweets_this_week += 1
				monday_to_sunday[datetime.datetime.weekday(tweet_time)] += 1
				twelve_am_to_eleven_pm[tweet_time.hour] += 1
		else:
			pass
	except Exception as e:
		pass
	user_data = {
		"image": user["profile_image_url_https"],
		"background_image": user["profile_background_image_url"],
		"verified": user["verified"],
		"statuses": user["statuses_count"],
		"screen_name": user["screen_name"],
		"display_name": user["name"],
		"description": user["description"],
		"location": user["location"],
		"url": user["url"],
		"date_joined": convert_twitter_datetime_to_string(user["created_at"]),
		"most_recent_post": most_recent_post,
		"statuses": user["statuses_count"],
		"following": user["friends_count"],
		"followers": user["followers_count"],
		# The number of Tweets this user has liked in the account’s lifetime
		"likes": user["favourites_count"],
		"lists": user["listed_count"],
		"twitter_user_id": user["id_str"],
		"tweet_language": tweet_language,
		"tweets_this_week": num_of_tweets_this_week,
		"retweet": int(num_of_retweets_by_user),
		"retweeted_count": int(retweeted),
		"tweets_by_day_of_week": monday_to_sunday,
		"tweets_by_hour_of_day": twelve_am_to_eleven_pm,
	}
	return (user_data)
get_details('BillGates')

{'image': 'https://pbs.twimg.com/profile_images/1564398871996174336/M-hffw5a_normal.jpg',
 'background_image': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'verified': True,
 'statuses': 4204,
 'screen_name': 'BillGates',
 'display_name': 'Bill Gates',
 'description': "Sharing things I'm learning through my foundation work and other interests.",
 'location': 'Seattle, WA',
 'url': 'https://t.co/UkvHzxDwkH',
 'date_joined': '2009-06-24 18:44:10',
 'most_recent_post': '2023-03-08 23:24:57',
 'following': 543,
 'followers': 62222113,
 'likes': 484,
 'lists': 121737,
 'twitter_user_id': '50393960',
 'tweet_language': 'en',
 'tweets_this_week': 22,
 'retweet_ratio': 8,
 'retweeted_count': 68092,
 'tweets_by_day_of_week': [16, 46, 47, 44, 29, 12, 6],
 'tweets_by_hour_of_day': [5,
  2,
  4,
  9,
  2,
  1,
  0,
  0,
  1,
  1,
  0,
  2,
  0,
  2,
  2,
  6,
  18,
  18,
  17,
  20,
  22,
  29,
  25,
  14]}

# fetch data by api  more than 24h

In [56]:
alldata=[]
nbr_req=0
data=pd.read_csv("./Data/cleandata.csv")
for index, row in data.iterrows():
    nbr_req += 1
    u = []
    u=get_details(row['screen_name'])
    if u!="suspended" :
        u['account_type']=row['account_type']
        alldata.append(u)
    
    if nbr_req==900:
        print('req = 900 please wait 15 minutes')
        nbr_req=0
        time.sleep(900)
        print('15 minutes')


req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes

# Save data in file

In [63]:
alldata
import json
jsonString = json.dumps(alldata)
with open("sample.json", "w") as outfile:
    json.dump(alldata, outfile)

read data from mongo

In [18]:

# # jsonString = data.to_json()
# myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# db = myclient.twitter
# collection = db.data
# x=collection.find({},{'_id': 0})
# x


,image,background_image,verified,statuses,screen_name,display_name,description,location,url,date_joined,...,likes,lists,twitter_user_id,tweet_language,tweets_this_week,retweet_ratio,retweeted_count,tweets_by_day_of_week,tweets_by_hour_of_day,account_type
0,https://pbs.twimg.com/profile_images/145899544...,http://abs.twimg.com/images/themes/theme11/bg.gif,False,7098,paty_castroo,Patrícia Castro,,"Maringá, Brasil",None,2009-07-04 22:41:51,...,12213,2,53779179,,0,0,0,"[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",bot
1,https://pbs.twimg.com/profile_images/632916759...,http://abs.twimg.com/images/themes/theme3/bg.gif,False,270,CBirckner,Carli Birckner,Television producer. Emmy Award winner. Disney...,"Baltimore, MD",None,2010-01-17 22:54:19,...,1484,2,105916557,en,0,16,103418,"[9, 4, 12, 15, 5, 0, 3]","[1, 4, 5, 0, 1, 0, 1, 1, 0, 0, 4, 0, 0, 0, 3, ...",human
2,https://pbs.twimg.com/profile_images/158696080...,http://abs.twimg.com/images/themes/theme1/bg.png,False,14302,amf_jay,JD,self discovery 💡 🧠,,None,2012-03-01 06:05:32,...,2958,2,509788597,en,0,77,867392,"[52, 14, 19, 24, 24, 29, 32]","[25, 15, 9, 20, 6, 10, 2, 9, 1, 1, 3, 0, 3, 1,...",human
3,https://pbs.twimg.com/profile_images/977012905...,http://abs.twimg.com/images/themes/theme6/bg.gif,False,4585,SaraCavolo,Sara Cavolo,,"Brooklyn, NY",None,2009-09-01 04:52:30,...,7983,3,70601327,en,0,54,2996382,"[19, 34, 42, 41, 28, 17, 15]","[10, 17, 18, 5, 1, 0, 0, 0, 1, 0, 1, 2, 7, 18,...",human
4,https://pbs.twimg.com/profile_images/668449819...,http://abs.twimg.com/images/themes/theme14/bg.gif,True,15851,DavidHenaoModel,David Henao,Productor de Televisión - Embajador de @Tienda...,"Miami, FL",https://t.co/KPAyvqMibx,2010-01-27 17:17:23,...,20105,172,108999927,und,0,10,66477,"[33, 23, 25, 24, 34, 28, 33]","[7, 17, 5, 3, 1, 0, 0, 2, 0, 0, 2, 6, 14, 14, ...",human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34042,https://pbs.twimg.com/profile_images/712018970...,http://abs.twimg.com/images/themes/theme14/bg.gif,True,60100,AshAlexiss,Ashley Alexiss,"Self Made Plus Model, #BeautyIsNotASize, CEO @...","Los Angeles, CA",https://t.co/7nNQF782OV,2010-12-07 08:11:16,...,61532,876,223764929,en,8,47,1375535,"[28, 33, 25, 31, 22, 28, 33]","[7, 18, 13, 13, 16, 9, 6, 12, 6, 5, 1, 0, 0, 1...",human
34043,https://pbs.twimg.com/profile_images/141825399...,http://abs.twimg.com/images/themes/theme9/bg.gif,True,40277,ahmemis,Ahmet Memiş,Gazeteci/Journalist,,https://t.co/0wll8Iymf4,2010-12-04 13:17:22,...,1,182,222789645,tr,0,0,15350,"[21, 67, 36, 18, 22, 18, 18]","[5, 0, 0, 0, 0, 1, 0, 2, 1, 6, 7, 9, 3, 7, 4, ...",human
34044,https://pbs.twimg.com/profile_images/142309872...,http://abs.twimg.com/images/themes/theme10/bg.gif,False,6679,Yuvannamontalvo,Yuvanna Montalvo,Modelo y Actriz Venezolana/ Venezuelan Model &...,,None,2009-10-06 23:53:30,...,28,372,80438218,zxx,0,8,18899,"[25, 23, 26, 42, 29, 31, 24]","[23, 26, 21, 8, 1, 2, 0, 0, 0, 2, 0, 0, 4, 10,...",human
34045,https://pbs.twimg.com/profile_images/157735786...,http://abs.twimg.com/images/themes/theme1/bg.png,True,2580,IAmVarunTej,Varun Tej Konidela,Indian!...Actor!...Need no more!,Hyderabad,https://t.co/kJoeCCukod,2013-03-24 21:04:16,...,871,378,1296642259,en,1,4,145792,"[23, 22, 20, 85, 20, 14, 16]","[0, 2, 1, 26, 25, 17, 9, 7, 7, 7, 12, 5, 24, 6...",human
